In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# 1. 샘플 데이터프레임 (score = 0 또는 1)
data = {
    "age": [22, 25, 30, 35, 40, 45, 50, 55, 60, 65],
    "gender": ["M", "F", "M", "F", "M", "F", "M", "F", "M", "F"],
    "score": [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # 이진 클래스
}
df = pd.DataFrame(data)

# 2. 전처리
df["gender"] = df["gender"].map({"M": 0, "F": 1})
df["age"] = df["age"] / 100.0  # 스케일링

x = torch.tensor(df[["age", "gender"]].values, dtype=torch.float32)
y = torch.tensor(df[["score"]].values, dtype=torch.float32)

In [ ]:
# 3. 모델 정의
class BinaryNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))  # 마지막만 sigmoid
        return x

In [ ]:
# 4. 모델, 손실함수, 옵티마이저
model = BinaryNN()
criterion = nn.BCELoss()  # 이진 분류용 Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [ ]:
# 5. 학습 루프
epochs = 100
for epoch in range(epochs):
    y_pred = model(x)
    loss = criterion(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {loss.item():.6f}")

# 6. 결과 확인
with torch.no_grad():
    y_prob = model(x)
    y_pred_class = (y_prob > 0.5).float()

print("\n예측 확률:")
print(y_prob)
print("\n예측 클래스:")
print(y_pred_class)

Epoch [10/100] | Loss: 0.673249
Epoch [20/100] | Loss: 0.619989
Epoch [30/100] | Loss: 0.364632
Epoch [40/100] | Loss: 0.178782
Epoch [50/100] | Loss: 0.110668
Epoch [60/100] | Loss: 0.078594
Epoch [70/100] | Loss: 0.059586
Epoch [80/100] | Loss: 0.048985
Epoch [90/100] | Loss: 0.040105
Epoch [100/100] | Loss: 0.034022

예측 확률:
tensor([[0.0029],
        [0.9379],
        [0.0029],
        [0.9379],
        [0.0029],
        [0.9379],
        [0.0029],
        [0.9379],
        [0.0030],
        [0.9379]])

예측 클래스:
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.]])


In [ ]:
#################################3
### 추론
################################
import torch

# 추론용 새 입력 데이터 (나이, 성별)
# 주의: gender는 학습 때처럼 M=0, F=1 로 인코딩
new_data = [
    {"age": 32, "gender": "M"},
    {"age": 50, "gender": "F"},
    {"age": 28, "gender": "F"},
]

#  gender → 숫자 변환 (훈련과 동일 규칙)
gender_map = {"M": 0, "F": 1}
for d in new_data:
    d["gender"] = gender_map[d["gender"]]

#  나이 정규화 (훈련과 동일하게 /100.0)
x_new = torch.tensor(
    [[d["age"] / 100.0, d["gender"]] for d in new_data],
    dtype=torch.float32
)

#  추론
model.eval()
with torch.no_grad():
    y_prob = model(x_new)           # sigmoid 확률 출력 (0~1)
    y_pred = (y_prob > 0.5).float() # 0.5 기준 이진 분류

print("입력 데이터:")
print(x_new)
print("\n예측 확률:")
print(y_prob)
print("\n예측 클래스 (0=Negative, 1=Positive):")
print(y_pred)


입력 데이터:
tensor([[0.3200, 0.0000],
        [0.5000, 1.0000],
        [0.2800, 1.0000]])

예측 확률:
tensor([[0.0029],
        [0.9379],
        [0.9379]])

예측 클래스 (0=Negative, 1=Positive):
tensor([[0.],
        [1.],
        [1.]])


In [ ]:


import torch

# gender 인코딩 규칙 (훈련 시와 동일하게)
GENDER_MAP = {"M": 0, "F": 1}

def predict(model, data_list):
    """
    추론 함수
    ----------------
    model      : 학습된 PyTorch 모델
    data_list  : [{'age': 30, 'gender': 'M'}, ...] 형식의 입력 리스트
    ----------------
    반환값: (y_prob, y_pred)
    """
    # 1. gender 숫자 변환 및 age 정규화 (/100.0)
    x_new = torch.tensor(
        [[d["age"] / 100.0, GENDER_MAP[d["gender"]]] for d in data_list],
        dtype=torch.float32
    )

    # 2. 모델 추론
    model.eval()
    with torch.no_grad():
        y_prob = model(x_new)            # sigmoid 확률값 (0~1)
        y_pred = (y_prob > 0.5).float()  # 0 또는 1로 변환

    return y_prob, y_pred


# 예시 ----------------------------------------------------
# 새로운 입력 데이터
new_samples = [
    {"age": 28, "gender": "F"},
    {"age": 45, "gender": "M"},
    {"age": 60, "gender": "F"}
]

# 추론 실행
y_prob, y_pred = predict(model, new_samples)

print("입력 데이터:")
print(new_samples)
print("\n예측 확률:")
print(y_prob)
print("\n예측 클래스:")
print(y_pred)


입력 데이터:
[{'age': 28, 'gender': 'F'}, {'age': 45, 'gender': 'M'}, {'age': 60, 'gender': 'F'}]

예측 확률:
tensor([[0.9379],
        [0.0029],
        [0.9379]])

예측 클래스:
tensor([[1.],
        [0.],
        [1.]])
